In [1]:
from llmtosql.model import WikiSQLModel
from llmtosql.trainer import Trainer
from llmtosql.dataloader import WikiSQLDataset
from llmtosql.utils.utils import plot_history, load_model, load_history
from matplotlib import pyplot as plt
import sys
import os
import json
import numpy as np
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
# os.environ['PYTORCH_ENABLE_MPS_FALLBACK']='1'
sys.path.insert(0, '..')

In [2]:
print(os.environ["VIRTUAL_ENV"])

/Users/albertomerango/Coeuraj/v_env


In [3]:
path = '../WikiSQL/data/dev.jsonl'

In [4]:
train_set =  WikiSQLDataset(type='dev')

In [5]:
# val_set = WikiSQLDataset(type='dev')

In [6]:
len(train_set.data)

8421

In [7]:
# datasets = (train_set, val_set)

In [8]:
model = WikiSQLModel(base_model_type='bert-base-cased')
try:
    model = load_model(model, 'model_output/model.pth')
except:
    print('No model loaded')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
config = {
    'seed': 32,
    'scheduler': None,
    'optimizer': 'adam',
    'momentum': 0.9,
    'weight_decay': 0.0,
    'lr': 0.001,
    'criterion': 'cross_entropy',
    'metric': 'accuracy',
    'pred_function': 'softmax',
    'model_dir': 'model_output',
    'backend': 'smddp'
}

In [10]:
trainer = Trainer(model, datasets=train_set, epochs=6, batch_size=16, is_parallel=False, **config)

2023-02-25 18:45.06 [info     ] Config inputs.                 config={'seed': 32, 'scheduler': None, 'optimizer': 'adam', 'momentum': 0.9, 'weight_decay': 0.0, 'lr': 0.001, 'criterion': 'cross_entropy', 'metric': 'accuracy', 'pred_function': 'softmax', 'model_dir': 'model_output', 'backend': 'smddp'}
2023-02-25 18:45.06 [info     ] Loading the model.
2023-02-25 18:45.06 [info     ] Training on device: cpu.
2023-02-25 18:45.06 [info     ] Loading training and validation set.
2023-02-25 18:45.06 [info     ] Preparing the data.
2023-02-25 18:45.06 [debug    ] Processes 8421/8421 (100%) of train data
2023-02-25 18:45.06 [debug    ] Processes 8421/8421 (100%) of validation data


In [11]:
trainer.fit()

2023-02-25 18:45.06 [info     ] Start training..
2023-02-25 18:45.06 [info     ] ------------------------------ EPOCH 1 / 6 ------------------------------


100%|██████████| 527/527 [2:04:41<00:00, 14.20s/batch, loss=17.9, metric=0.161]   

2023-02-25 23:08.15 [info     ] Saving the model.


RuntimeError: Parent directory model_output does not exist.

In [ ]:
trainer.save_history_('model_output')

In [ ]:
history = load_history('model_output')

In [ ]:
plot_history(history)

In [ ]:
model = MLModel()
model = load_model(model, 'model_output/model.pth')
test_loss, test_accuracy = trainer.test(model, trainer.val_loader)

In [ ]:
print(f'Model accuracy on test: {test_accuracy}')